- 2003: 1891 rows
- 2006: 1931 rows
- 2010: 1920 rows
- 2013: 1951 rows
- 2016: 2016 rows
- 2021: 2010 rows
- 2024: 1798 rows

In [1]:
import os
import gc
import json
import rasterio
import numpy as np
import pandas as pd
from tqdm import tqdm
import geopandas as gpd
from pathlib import Path
from osgeo import gdal, osr
import dask_geopandas as dgpd
from shapely.geometry import shape
from rasterstats import zonal_stats

BASE_DIR = Path('/Users/wenlanzhang/PycharmProjects/Mapineq/src/data-wrangling/')
DATA_DIR = Path('/Users/wenlanzhang/Downloads/PhD_UCL/Data/Oxford')

2020-2021 （NUT2006）: 3862  
2020 (NUTS2006): 1931  
2020-2021 (NUTS 2003, 2006)

In [11]:
df = pd.read_csv('/Users/wenlanzhang/Downloads/VIIRS_NUTS_All_19_21 (1).csv')
# df = df.drop(columns=['system:index', 'CNTR_CODE', 'LEVL_CODE', 'NUTS_ID', 'NUTS_NAME', 'min'])
df

,system:index,geo_source,max,mean,obsTime,stdDev,.geo
0,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_2_0000...,NUTS2003,371.416779,0.700943,2019,3.080153,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_2_0000...,NUTS2003,371.416779,0.562128,2019,2.892622,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_2_0000...,NUTS2003,174.950760,1.227383,2019,4.131787,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_2_0000...,NUTS2003,177.478455,0.398776,2019,1.858099,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_2_0000...,NUTS2003,46.541229,0.807894,2019,2.226023,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...,...,...,...,...
40546,2_0000000000000000052e,NUTS2024,25.373180,0.554214,2021,1.470890,"{""type"":""MultiPoint"",""coordinates"":[]}"
40547,2_0000000000000000052f,NUTS2024,15.038698,0.351601,2021,0.902235,"{""type"":""MultiPoint"",""coordinates"":[]}"
40548,2_00000000000000000532,NUTS2024,27.257809,0.400070,2021,1.318365,"{""type"":""MultiPoint"",""coordinates"":[]}"
40549,2_000000000000000004d6,NUTS2024,46.032799,0.432952,2021,1.798095,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [13]:
df2 = pd.read_csv('/Users/wenlanzhang/Downloads/VIIRS_NUTS_All_19_21.csv')
# df = df.drop(columns=['system:index', 'CNTR_CODE', 'LEVL_CODE', 'NUTS_ID', 'NUTS_NAME', 'min'])
df2

,system:index,NUTS_ID,geo_source,max,mean,obsTime,stdDev,.geo
0,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_2_0000...,AT,NUTS2003,371.416779,0.700943,2019,3.080153,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_2_0000...,AT3,NUTS2003,371.416779,0.562128,2019,2.892622,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_2_0000...,AT1,NUTS2003,174.950760,1.227383,2019,4.131787,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_2_0000...,AT2,NUTS2003,177.478455,0.398776,2019,1.858099,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_2_0000...,AT34,NUTS2003,46.541229,0.807894,2019,2.226023,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...,...,...,...,...,...
40546,2_0000000000000000052e,SI037,NUTS2024,25.373180,0.554214,2021,1.470890,"{""type"":""MultiPoint"",""coordinates"":[]}"
40547,2_0000000000000000052f,SI038,NUTS2024,15.038698,0.351601,2021,0.902235,"{""type"":""MultiPoint"",""coordinates"":[]}"
40548,2_00000000000000000532,SI043,NUTS2024,27.257809,0.400070,2021,1.318365,"{""type"":""MultiPoint"",""coordinates"":[]}"
40549,2_000000000000000004d6,SK041,NUTS2024,46.032799,0.432952,2021,1.798095,"{""type"":""MultiPoint"",""coordinates"":[]}"


# Export with geometry to Check in QGIS

In [3]:
# # Drop the 'system:index' column
# if 'system:index' in df.columns:
#     df = df.drop(columns=['system:index'])

# # Convert .geo (GeoJSON) to WKT
# def geojson_to_wkt(geo_str):
#     try:
#         geom = shape(json.loads(geo_str))
#         return geom.wkt
#     except Exception as e:
#         print(f"Error parsing geometry: {e}")
#         return None

# df['WKT'] = df['.geo'].apply(geojson_to_wkt)

# # Drop the original .geo column
# df = df.drop(columns=['.geo'])
# df

,NUTS_ID,geo_source,max,mean,stdDev,year,WKT
0,AT124,NUTS2006,39.095600,0.485493,1.325237,2020,"POLYGON ((15.5423568687789 48.90795874625594, ..."
1,AT125,NUTS2006,52.678535,0.746196,1.576954,2020,POLYGON ((15.753875401943136 48.85239782145810...
2,AT313,NUTS2006,25.432070,0.314568,0.894521,2020,POLYGON ((13.839701890307119 48.77145300222119...
3,AT126,NUTS2006,68.902916,1.495743,2.753673,2020,POLYGON ((15.731365711283962 48.38419992107961...
4,AT311,NUTS2006,62.645573,0.509443,1.775113,2020,POLYGON ((13.727576738328024 48.51302851665202...
...,...,...,...,...,...,...,...
7639,UKC12,NUTS2003,111.081902,9.822134,15.013664,2021,POLYGON ((-0.7936467003412386 54.5584462763246...
7640,UKC11,NUTS2003,92.253082,10.010130,13.173835,2021,MULTIPOLYGON (((-1.2347878213391148 54.5103682...
7641,UKN03,NUTS2003,46.515072,1.200607,2.830120,2021,MULTIPOLYGON (((-5.641572728675446 54.41375189...
7642,UKE12,NUTS2003,280.591919,1.421410,5.261583,2021,POLYGON ((-1.048596355843249 53.65608239478745...


In [4]:
# df_test = df[df['LEVL_CODE'] == 2]
# df_test.to_csv("/Users/wenlanzhang/Downloads/formatted_for_qgis.csv", index=True, index_label="id")

In [ ]:
# # Drop the specified columns
# df = df.drop(columns=['WKT'])

# Export

In [5]:
df = df.drop(columns=['.geo'])
df

,geo_source,max,mean,obsTime,stdDev
0,NUTS2003,371.416779,0.700943,2019,3.080153
1,NUTS2003,371.416779,0.562128,2019,2.892622
2,NUTS2003,174.950760,1.227383,2019,4.131787
3,NUTS2003,177.478455,0.398776,2019,1.858099
4,NUTS2003,46.541229,0.807894,2019,2.226023
...,...,...,...,...,...
40546,NUTS2024,25.373180,0.554214,2021,1.470890
40547,NUTS2024,15.038698,0.351601,2021,0.902235
40548,NUTS2024,27.257809,0.400070,2021,1.318365
40549,NUTS2024,46.032799,0.432952,2021,1.798095


In [6]:
df

,geo_source,max,mean,obsTime,stdDev
0,NUTS2003,371.416779,0.700943,2019,3.080153
1,NUTS2003,371.416779,0.562128,2019,2.892622
2,NUTS2003,174.950760,1.227383,2019,4.131787
3,NUTS2003,177.478455,0.398776,2019,1.858099
4,NUTS2003,46.541229,0.807894,2019,2.226023
...,...,...,...,...,...
40546,NUTS2024,25.373180,0.554214,2021,1.470890
40547,NUTS2024,15.038698,0.351601,2021,0.902235
40548,NUTS2024,27.257809,0.400070,2021,1.318365
40549,NUTS2024,46.032799,0.432952,2021,1.798095


In [7]:
# Rename 'stdDev' to 'std_dev' and 'year' to 'obs_time'
df = df.rename(columns={'stdDev': 'std_dev', 'year': 'obsTime'})

# Melt the dataframe to long format
df_long = pd.melt(df, 
                  id_vars=['geo_source', 'obsTime'], 
                  value_vars=['max', 'mean', 'std_dev'],
                  var_name='metric', 
                  value_name='obsValue')

cols = [col for col in df_long.columns if col != 'geo_source'] + ['geo_source']
df_long = df_long[cols]
df_long

,obsTime,metric,obsValue,geo_source
0,2019,max,371.416779,NUTS2003
1,2019,max,371.416779,NUTS2003
2,2019,max,174.950760,NUTS2003
3,2019,max,177.478455,NUTS2003
4,2019,max,46.541229,NUTS2003
...,...,...,...,...
121648,2021,std_dev,1.470890,NUTS2024
121649,2021,std_dev,0.902235,NUTS2024
121650,2021,std_dev,1.318365,NUTS2024
121651,2021,std_dev,1.798095,NUTS2024


In [6]:
# df_long.to_csv("/Users/wenlanzhang/Downloads/formatted_output.csv", index=True, index_label="id")

In [46]:
# Define the years you're interested in
years = [2003, 2006, 2010, 2013, 2016, 2021, 2024]

# Loop over each year, read the file, and print the number of rows
for year in years:
    file_path = DATA_DIR / f"NUTS/NUTS_RG_01M_{year}_3035.geojson"
    try:
        gdf = gpd.read_file(file_path)
        print(f"{year}: {len(gdf)} rows")
    except Exception as e:
        print(f"Failed to read {file_path.name}: {e}")

2003: 1891 rows
2006: 1931 rows
2010: 1920 rows
2013: 1951 rows
2016: 2016 rows
2021: 2010 rows
2024: 1798 rows
